In [ ]:
import os
import pandas as pd
import numpy as np
import json
import difflib as dl
import funcTest as ft

In [ ]:
os.chdir('F:\\TFG\\datasets\\nature-dataset')

# loading the events data
events={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open('./events/events_%s.json' %nation) as json_data:
        events[nation] = json.load(json_data)
        
# loading the match data
matches={}
nations = ['Italy','England','Germany','France','Spain','European_Championship','World_Cup']
for nation in nations:
    with open('./matches/matches_%s.json' %nation) as json_data:
        matches[nation] = json.load(json_data)

# loading the players data
players={}
with open('./players.json') as json_data:
    players = json.load(json_data)
    
# loading the teams data
teams={}
with open('./teams.json',encoding='utf-8') as json_data:
    teams = json.load(json_data)

# loading the competitions data
competitions={}
with open('./competitions.json') as json_data:
    competitions = json.load(json_data)
    
# loading the playerank data
playerank={}
with open('./playerank.json') as json_data:
    playerank = json.load(json_data)
    
# loading the referees data
# referees={}
# with open('./referees.json') as json_data:
#     referees = json.load(json_data)

# loading the coaches data
coaches={}
with open('./coaches.json') as json_data:
    coaches = json.load(json_data)

In [ ]:
matches['Spain'][0].keys()

<h3>Vamos a concatenar todos los partidos de las 5 ligas</h3>

In [ ]:
partidos = []

paises = ['Italy','England','Germany','France','Spain']

for pais in paises:
    for partido in matches[pais]:
        partidos.append(partido)
        
partidos = np.array(partidos,dtype=dict)
partidos.shape

<h3>Crear tabla de partidos y filtrar atributos</h3>

<h4>Primero, agrupamos los datos que deseamos para los datasets</h4>

<b>Dataset 1</b>: será un dataset más simple simplemente con resultados  y IDs de equipos, competiciones, temporadas y nº de jornada

    - ID Partido
    - ID Competicion
    - ID Temporada
    - Jornada (roundID)
    - IDs partidos para cada equipo -> se calcula
    - Fecha (dateutc) -> Formato YYYY-MM-DD hh:mm:ss
    - ID Equipos
    - Nombre Equipos
    - Goles Equipos
    - ID Ganador o 0 si empate
    - Formaciones Equipos (si tienen 1 sino 0)

In [ ]:
pd.DataFrame(partidos)
s = pd.Series(partidos[0])
s.index

In [ ]:
col = ['a',
       'b']

In [ ]:
match_serie = [pd.Series(p) for p in partidos]
# print(match_serie)
rawDB = pd.DataFrame(match_serie, columns=(match_serie[0]).index)

columnNamesDB = ['wyId','competitionId','seasonId','roundId','venue','dateutc','winner'] # wyId = ID del partido en el DB
rawDB = rawDB[columnNamesDB]

In [ ]:
rawDB

<b>Añadimos ahora los datos de cada equipo:</b>

Para añadirlos hay que tener en cuenta que los datos de cada equipo se encuentran como un objeto JSON dentro del JSON del partido, es decir dentro del atributo <b>teamsData</b>

Dentro del atributo teamsData encontramos dos JSON, uno para cada equipo. El índice de cada JSON es el Id del equipo

In [ ]:
teamsData = [list(m['teamsData'].values()) for m in match_serie] # partido x equipo
pd.Series(teamsData[0][0])['formation']

In [ ]:
teamsData = [list(m['teamsData'].values()) for m in match_serie] # partido x equipo

homeT = []
awayT = []

for m in teamsData:
#     print(m[0]['teamId'])
    if m[0]['side'] == 'home':
        homeT.append(pd.Series(m[0]))
        awayT.append(pd.Series(m[1]))
    else:
        homeT.append(pd.Series(m[1]))
        awayT.append(pd.Series(m[0]))
           
# print(set([x['teamId'] for x in homeT]))

# debemos cambiar el nombre de las columnas para que no se llamen igual las Home que las Away

colH = [str(x) + '_home' for x in homeT[0].index]
colA = [str(x) + '_away' for x in awayT[0].index]

hData = pd.DataFrame(homeT, columns=homeT[0].index)
aData = pd.DataFrame(awayT, columns=awayT[0].index)
hData.columns = colH
aData.columns = colA

# Eliminamos los atributos que no nos interesan


# Añadimos el nombre del equipo -> hay que buscarlo en el JSON de equipos

# homeNames = [teams['wyId'==hT['teamId']]['name'] for hT in homeT]
# awayNames = [teams['wyId'==aT['teamId']]['name'] for aT in awayT]
# print(homeNames[500])

# hData = hData.insert(len(hData.columns),)

assert len(hData) == len(aData) == len(rawDB)

In [ ]:
hData[:3]

In [ ]:
rawDB = pd.concat([rawDB,hData,aData],axis=1)

In [ ]:
rawDB['formation_home'] = np.array(rawDB['formation_home'],dtype=dict)
rawDB['formation_away'] = np.array(rawDB['formation_away'],dtype=dict)

In [ ]:
# rawDB.to_csv('F:\\TFG\\datasets\\data_train\\rawDB.csv',index=False)

In [ ]:
rawDB

In [ ]:
rawDB['formation_home'][0]

In [ ]:
rawDB.iloc[0]

Ahora deberemos seleccionar y arreglar los datos que queremos

In [ ]:
rawDB.columns

In [ ]:
rawDB = rawDB[[
    'wyId', 'competitionId', 'seasonId', 'roundId', 'dateutc', 'winner',
    'teamId_home','score_home','scoreHT_home','teamId_away', 'score_away',
    'scoreHT_away'
     ]]

assert len(rawDB) == 1826 == len(set(rawDB['wyId']))

rawDB.loc[360]

In [ ]:
local = []
visit = []

for row in rawDB['teamId_home']:
    for t in teams:
        local.append(t['name']) if t['wyId']==row else ''
        
for row in rawDB['teamId_away']:
    for t in teams:
        visit.append(t['name']) if t['wyId']==row else ''

In [ ]:
assert len(local) == len(visit) == 1826

In [ ]:
# Sacar nombre de los equipos

for t in  teams:
    print(t['name']) if t['wyId']==3161 else ''
        

<h3>Datasets que vamos a hacer</h3>

    - Cuotas con Victoria local (1) o otro resultado (0) -> cuotas_2
    - Cuotas con Victoria local (-1) empate (0) o victoria visitante (1) -> cuotas_3
    - Cuotas binaria + ultimos N partidos -> cuotas_racha_N
    - Cuotas binaria + ultimos N partidos + goles N partidos -> cuotas_racha_goles_N
    - Otras: 
        - teniendo en cuenta clasificaciones pasadas
        - teniendo en cuenta formación
        - teniendo en cuenta 

<b>Añadimos al dataset global las cuotas</b>

Habrá que identificar cada partido en ambas bases de datos

In [ ]:
os.chdir('F:\\TFG\\datasets\\football-data\\')
dir_footdata = os.getcwd()

In [ ]:
      
#LALIGA
data_fd = {}
data_fd['Esp'] = pd.read_csv(dir_footdata +'\\SP1_'+ '1718' +'.csv')
       
#PREMIER
data_fd['Ing'] = pd.read_csv(dir_footdata +'\\E0_'+ '1718' +'.csv')

#BUNDESLIGA
data_fd['Ale'] = pd.read_csv(dir_footdata +'\\D1_'+ '1718' +'.csv')

#LIGUE 1
data_fd['Fra'] = pd.read_csv(dir_footdata +'\\F1_'+ '1718' +'.csv')

#SERIE A
data_fd['Ita'] = pd.read_csv(dir_footdata +'\\I1_'+ '1718' +'.csv')


In [ ]:
type(data_fd['Esp'].B365H[0])

In [ ]:
data_fd['Esp'][data_fd['Esp'].columns[:25]]

In [ ]:
fdataDB = pd.concat([data_fd['Esp'],data_fd['Ing'],data_fd['Ale'],data_fd['Fra'],data_fd['Ita']],ignore_index=True)
fdataDB[:4]

In [ ]:
fdataDB.columns

In [ ]:
# fdataDB.to_csv('F:\\TFG\\datasets\\data_train\\fdataDB.csv',index=False)

In [ ]:
fdataDB = fdataDB[['Div','Date','HomeTeam','AwayTeam','FTHG','FTAG', 'FTR','HS',
                   'AS','HST','AST','HC','AC','HF','AF','B365H', 'B365D', 'B365A']]
fdataDB.loc[[0,100,400,700,1000,1400,1700]]

assert len(fdataDB) == len(rawDB)

Hay que identificar los equipos para identificar los partidos

In [ ]:
fdTeams = set(fdataDB['HomeTeam'])

# equipos nature dataset

naTeams = [t['name'] for t in teams if t['type']=='club']

assert len(fdTeams)==len(naTeams)==98

In [ ]:
iguales = [t for t in fdTeams if t in naTeams]
len(iguales) # tenemos que 58 clubs se llaman igual en ambos datasets

desiguales = [t for t in naTeams if t not in iguales]

In [ ]:
len(iguales)

In [ ]:
encontrados = []
no_encontrados = []

for des in desiguales:
    find = dl.get_close_matches(des,fdTeams)
    if len(find)>0:
        encontrados.append((des,find[0]))
    else:
        no_encontrados.append(des)
        

In [ ]:
len(encontrados)
encontrados # (na, fd)

In [ ]:
emparejadosFD = iguales
emparejadosNA = iguales

for (x,y) in encontrados:
    emparejadosFD.append(y)
    emparejadosNA.append(x)
    
desigualesFD = [t for t in fdTeams if t not in emparejadosFD]
desigualesNA = [t for t in naTeams if t not in emparejadosNA]

In [ ]:
assert len(desigualesFD) == 12 == len(desigualesNA)

desigualesNA

In [ ]:
desigualesFD

In [ ]:
encontrados

In [ ]:
aux_equipos = ['Celta','Alaves','Ath Bilbao','Lyon','Paris SG','Spal','Brighton','Inter',
               'St Etienne','FC Koln','La Coruna','Malaga']

equipos = {} # { nature_data : football_data } }

for t in iguales:
    equipos[t] = t

for x,y in encontrados:
    equipos[x] = y
    
for x,y in zip(desigualesNA,aux_equipos):
    equipos[x] = y
    
# bug: los equipos 'encontrados' salen dupplicados?
for _,y in encontrados:
    if y in equipos.keys():
        del(equipos[y]) 
        
assert len(equipos) == 98

In [ ]:
# comprobamos que todos los elementos coinciden

x = 0

for e in equipos.keys():
    x = x + naTeams.count(e)

assert x == len(naTeams) == len(equipos.keys())

In [ ]:
equipos['Deportivo La Coru\\u00f1a']

Ahora debemos encontrar asociar los equipos de football_data con los de nature_data

In [ ]:
nat_ID_Names = {}

for t in teams:
    if t['type'] == 'club':
        nat_ID_Names[t['wyId']] = t['name']
        
assert len(nat_ID_Names.keys()) == 98 == len(equipos.keys())

aux = equipos
equipos = {}

equipos['equipos'] = aux
equipos['nat_ID_Names'] = nat_ID_Names

with open('F:\\TFG\\datasets\\data_train\\equipos.json','w') as js:
    json.dump(equipos,js)

In [ ]:
rawDB

<b> Vamos a añadir ya las cuotas al dataset de Nature que hemos creado anteriormente </b>

Para ello debemos iterar sobre el dataframe de Nature y buscar en el dataset de Football-Data que partido coincide con esos rivales.

(Hay que tener en cuenta que si tuviesemos más de temporadas se repetiria un mismo enfrentamiento entre dos equipos distintos)

In [ ]:
rawDB[:3]

In [ ]:
fdataDB[:3]

In [ ]:
len(rawDB)

In [ ]:
# Tarda mucho en ejecutarse

data_cuotas = ['HS','AS','HST','AST','HC','AC','HF','AF','B365H', 'B365D', 'B365A']

cuotaNatDB = rawDB
cuotasFeat = {}

for dc in data_cuotas:
    cuotasFeat[dc] = np.empty(shape=(len(rawDB)),dtype=object)
    
index = 0

for h,a in zip(rawDB['teamId_home'],rawDB['teamId_away']):
    for i in range(len(rawDB)):
        p = fdataDB.loc[i]
        h_name = equipos[nat_ID_Names[h]]
        a_name = equipos[nat_ID_Names[a]]
        if p['HomeTeam']==h_name and p['AwayTeam']==a_name:
            for dc in data_cuotas:
                cuotasFeat[dc].put(index,p[dc])
            index += 1
            break

for dc in data_cuotas:
    assert len(cuotasFeat[dc]) == len(rawDB)
    cuotaNatDB[dc] = cuotasFeat[dc]

# # Tarda mucho en ejecutarse

# bet_h_nat = []
# bet_a_nat = []

# for h,a in zip(rawDB['teamId_home'],rawDB['teamId_away']):
#     for i in range(1826):
#         p = fdataDB.loc[i]
#         h_name = equipos[nat_ID_Names[h]]
#         a_name = equipos[nat_ID_Names[a]]
#         if p['HomeTeam']==h_name and p['AwayTeam']==a_name:
#             bet_h_nat.append(p['B365H'])
#             bet_a_nat.append(p['B365A'])
#             break

# assert len(bet_h_nat) == len(bet_a_nat) == 1826

# cuotaNatDB = rawDB
# cuotaNatDB['B365H'] = bet_h_nat
# cuotaNatDB['B365A'] = bet_a_nat

In [ ]:
# EXPORTAMOS CSV
# cuotaNatDB.to_csv('F:\\TFG\\datasets\\data_train\\cuotaNatDB.csv',index=False)
# cuotaNatDB.to_csv('F:\\TFG\\datasets\\data_train\\Nature_wStatsDB_v1.csv',index=False)


# IMPORTAMOS CSV
cuotaNatDB = pd.read_csv('F:\\TFG\\datasets\\data_train\\cuotaNatDB.csv')
statsNatDB_v1 = pd.read_csv('F:\\TFG\\datasets\\data_train\\Nature_wStatsDB_v1.csv')

In [ ]:
statsNatDB_v1

Creamos la base de datos para que la puede usar el Modelo

- -1 gana local
- 0 empate
- 1 gana visitante

In [ ]:
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotaNatDB

In [ ]:
data_cuotas

In [ ]:
cuotasDB = cuotaNatDB[data_cuotas]

In [ ]:
def calcRES(db):
    res = []
    for h,a in zip(db['score_home'],db['score_away']):
        if h>a:
            res.append(-1)
        elif a>h:
            res.append(1)
        else:
            res.append(0) 
    
    assert len(res) == len(db)
    
    return res

In [ ]:
# LA EXPORTAMOS
# cuotasDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB.csv',index=False)
# cuotaNatDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv',index=False)

# LO IMPORTAMOS
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotasDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB.csv')
cuotasDB[:5]

<b> Para el siguiente Dataset vamos a registrar la racha (resultados y goles) de los ultimos N partidos de un equipos </b>

In [ ]:
cuotaNatDB.loc[cuotaNatDB['competitionId'] == 524][:5]

In [ ]:
cuotaNatDB

In [ ]:
def upt_features(feat,pts,hs_,goals,hst_,hc_,hf_,i):
    feat['points'] += pts
    feat['goals'] += goals
    feat['shots'] += hs_
    feat['on_target'] += hst_
    feat['corners'] += hc_
    feat['fouls'] += hf_
    return i+1
    
def descartar(feat,i,n):
    print(i, feat['goals'])
    if i < n:
        print('Partidos nulos')
        feat['points'] = np.NaN
        feat['goals'] = np.NaN
        feat['shots'] = np.NaN
        feat['on_target'] = np.NaN
        feat['corners'] = np.NaN
        feat['fouls'] = np.NaN    
    
def crear_tendencias(n=5,solo_local=True):
    
    features = {'ptsH': [], 'ptsA' : [], 'goalsH' : [], 'goalsA' : [],
                'HS' : [], 'AS' : [], 'HST' : [], 'AST' : [], 'HC' : [], 'AC' : [], 'HF' : [], 'AF' : []}
    id_comp = list(set(statsNatDB_v1['competitionId']))
    aux_df = statsNatDB_v1

    for comp in id_comp:
        rows = statsNatDB_v1.index[statsNatDB_v1['competitionId']==comp]
        df_comp = aux_df[rows[0]:rows[-1]+1]


        for index in range(rows[0],rows[-1]+1):
    #         print(index)
            ht = df_comp['teamId_home'][index]
            at = df_comp['teamId_away'][index]
            len_H = 0
            len_A = 0
            
            featH = {'points' : 0, 'goals' : 0, 'shots' : 0, 'on_target' : 0, 'corners' : 0, 'fouls' : 0}
            featA = {'points' : 0, 'goals' : 0, 'shots' : 0, 'on_target' : 0, 'corners' : 0, 'fouls' : 0}

            for i in range(index+1,rows[-1]+1):
                p = df_comp.loc[i]  
                h_aux = p['teamId_home']
                if ht == h_aux:
                    if p['score_home'] == p['score_away']:
                        len_H = upt_features(featH,1,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                    elif p['score_home'] > p['score_away']:
                        len_H = upt_features(featH,3,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                    else:
                        len_H = upt_features(featH,0,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_H)
                elif ht == p['teamId_away'] and solo_local:
                    if p['score_home'] == p['score_away']:
                        len_H = upt_features(featH,1,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)
                    elif p['score_home'] > p['score_away']:
                        len_H = upt_features(featH,0,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)
                    else:
                        len_H = upt_features(featH,3,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_H)

                if len_H == n:
                    break
                
            descartar(featH,len_H,n)

            for i in range(index+1,rows[-1]+1):
                p = df_comp.loc[i]
                a_aux = p['teamId_home']
                if at == a_aux and solo_local:
                    if p['score_home'] == p['score_away']:
                        len_A = upt_features(featA,1,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                    elif p['score_home'] > p['score_away']:
                        len_A = upt_features(featA,3,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                    else:
                        len_A = upt_features(featA,0,p['HS'],p['score_home'],p['HST'],p['HC'],p['HF'],len_A)
                elif at == p['teamId_away']:
                    if p['score_home'] == p['score_away']:
                        len_A = upt_features(featA,1,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)
                    elif p['score_home'] > p['score_away']:
                        len_A = upt_features(featA,0,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)
                    else:
                        len_A = upt_features(featA,3,p['AS'],p['score_away'],p['AST'],p['AC'],p['AF'],len_A)

                if len_A == n:
                    break

            descartar(featA,len_A,n)
        
            print()
            
            features['ptsH'].append(featH['points'])
            features['goalsH'].append(featH['goals'])
            features['HS'].append(featH['shots'])
            features['HST'].append(featH['on_target'])
            features['HC'].append(featH['corners'])
            features['HF'].append(featH['fouls'])

            features['ptsA'].append(featA['points'])
            features['goalsA'].append(featA['goals'])
            features['AS'].append(featA['shots'])
            features['AST'].append(featA['on_target'])
            features['AC'].append(featA['corners'])
            features['AF'].append(featA['fouls'])
                
    assert len(features['ptsH']) == len(features['ptsA']) == len(cuotaNatDB)                    
                
    return features


In [ ]:
features = crear_tendencias(n=3)

In [ ]:
claves = []

for c in list(features.keys()):
    claves.append('tend_' + str(c))
    
claves

In [ ]:
tend_values = list(features.values())

In [ ]:
statsDB_tend3 = statsNatDB_v1

for i in range(0,len(tend_values)):
    statsDB_tend3[claves[i]] = tend_values[i]
    
statsDB_tend3['tend_AC']

Vamos a simplificar el dataset para poder ser usado

In [ ]:
res = calcRES(statsDB_tend3)
statsDB_tend3['res'] = res

In [ ]:
# LA EXPORTAMOS
# cuotasDB_tend5.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB_tend5.csv',index=False)
# cuotaNatDB.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv',index=False)
# statsDB_tend3.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\statsDB_tend3.csv',index=False)

# LO IMPORTAMOS
cuotaNatDB = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotaNatDB.csv')
cuotasDB_tend5 = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotas3DB_tend5.csv')
statsDB_tend3 = pd.read_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\statsDB_tend3.csv')
statsDB_tend3[:5]

In [ ]:
cuotasDB_tend5[:-5]

In [ ]:
cuotasDB_tend3 = cuotasDB
tendenciasH, tendenciasA = crear_tendencias(n=3)
cuotasDB_tend3['tendencia_H'] = tendenciasH
cuotasDB_tend3['tendencia_A'] = tendenciasA

cuotasDB_tend3.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotasDB_tend3.csv',index=False)

In [ ]:
d = cuotasDB_tend3
dif_tend = [y-x for x,y in zip(d['tendencia_H'],d['tendencia_A'])]
assert len(dif_tend) == len(d)

In [ ]:
cuotasDB_tend3_norm = cuotasDB_tend3
cuotasDB_tend3_norm['dif_tend'] = dif_tend
cuotasDB_tend3_norm = ft.normalDB(cuotasDB_tend3_norm)
cuotasDB_tend3_norm = cuotasDB_tend3_norm.drop(['tendencia_H', 'tendencia_A'], axis=1)
cuotasDB_tend3_norm.to_csv('F:\\2020-21\\Beca VRAIN\\datasets\\data_train\\cuotasDB_tend3_norm.csv',index=False)

In [ ]:
data = statsDB_tend3
data = data.dropna(axis=0)
data

dif_puntos = data['tend_ptsA'] - data['tend_ptsH']
dif_goles = data['tend_goalsA'] - data['tend_goalsH']
dif_shots = data['tend_AS'] - data['tend_HS']
dif_targets = data['tend_AST'] - data['tend_HST']
dif_corners = data['tend_AC'] - data['tend_HC']

data['dif_puntos'] = dif_puntos
data['dif_goles'] = dif_goles
data['dif_shots'] = dif_shots
data['dif_targets'] = dif_targets
data['dif_corners'] = dif_corners

Lo siguiente...

Intentar visualizar los datos y probar algun modelo con Scikit Learn

Quiero ver si hay alguna correlación entre ambas variables (cuotas, tendencias...), ya que de haberla, no tendria sentido usar ambas variables en el modelo.